# Impact of Various Factors on Asthma Rates in the United States

## Data Model

#### Import Packages and Load Data

In [1]:
import pandas as pd
import numpy as np
import math
from pandasql import sqldf

In [2]:
cdc = pd.read_csv('Data/CDC.csv', converters={'countyfips': str})
emissions = pd.read_csv('Data/Emissions.csv', converters={'state_code': str, 'county_code': str, 'parameter_code':str})
income = pd.read_csv('Data/Income.csv', converters={'GeoFips': str})

#### Transform Data and Create Master Files

Create State, County and GeoFips codes needed for aggregations and needed joins.

In [3]:
# Add 0's to County Code in emissions data set
def county_code(x):
    if len(x) == 1:
        return "00" + str(x)
    elif len(x) ==2:
        return "0" + str(x)
    else:
        return str(x)

# Apply function
emissions['county_code'] = emissions['county_code'].apply(county_code)

In [4]:
# geoFips for emissions
emissions['GeoFips'] = emissions['state_code'] + emissions['county_code']

In [6]:
len(emissions['GeoFips'].unique())

705

In [6]:
emissions.shape

(10926, 57)

Add "0" to GeoFips code in income dataset

In [7]:
# Add 0's to GeoFips Code in income data set
def geofips_code(x):
    if len(x) == 4:
        return "0" + str(x)
    else:
        return str(x)

# Apply function
income['GeoFips'] = income['GeoFips'].apply(geofips_code)

In this next step aggregate PM25 emissions (Quarterly Mean of Daily Average Mean) by each county.

In [8]:
# By County

# write query
q = """SELECT GeoFips, 
AVG(arithmetic_mean) as avg_emissions
FROM emissions
WHERE parameter_code = '88101' AND pollutant_standard = 'PM25 Annual 2012'
GROUP BY GeoFips
"""
pysqldf = lambda q: sqldf(q, globals())
county_df_emissions = pysqldf(q)

In [9]:
county_df_emissions.shape

(625, 2)

Because numerous County's are missing PM25 emissions data, aggregate PM25 emissions (Quarterly Mean of Daily Average Mean) by each State. These average State PM25 values will be inputted for County's where PM25 emissions are NA.

In [10]:
# By State

# write query
q = """SELECT state_code, 
AVG(arithmetic_mean) as avg_emissions
FROM emissions
WHERE parameter_code = '88101' AND pollutant_standard = 'PM25 Annual 2012'
GROUP BY state_code
"""
pysqldf = lambda q: sqldf(q, globals())
state_df_emissions = pysqldf(q)

In [11]:
state_df_emissions.head()

,state_code,avg_emissions
0,01,8.540471
1,02,9.153661
2,04,6.472957
3,05,8.653864
4,06,7.888009


Select relevent columns from the CDC data

In [12]:
cdc_analysis = cdc[['stateabbr', 'statedesc', 'countyname', 'countyfips', 'totalpopulation',
       'access2_adjprev', 'casthma_adjprev', 'csmoking_adjprev']]

Add State Codes and County Codes to each invidual data set

In [13]:
def state_code(x):
    if len(x) == 5:
        return str(x[0]) + str(x[1])
    elif len(x) ==4:
        return "0" + str(x[0])
    else:
        return str(x)

# State Codes
cdc_analysis['StateCode'] = cdc_analysis['countyfips'].apply(state_code)
county_df_emissions['StateCode'] = county_df_emissions['GeoFips'].apply(state_code)
income['StateCode'] = income['GeoFips'].apply(state_code)

#County Codes
cdc_analysis['CountyCode'] = cdc_analysis['countyfips'].str[-3:]
county_df_emissions['CountyCode'] = county_df_emissions['GeoFips'].str[-3:]
income['CountyCode'] = income['GeoFips'].str[-3:]

<ipython-input-13-cfd2d2e9d92f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_analysis['StateCode'] = cdc_analysis['countyfips'].apply(state_code)
<ipython-input-13-cfd2d2e9d92f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_analysis['CountyCode'] = cdc_analysis['countyfips'].str[-3:]


Create master data final

In [14]:
# write query
q = """SELECT cdc_analysis.*, 
county_df_emissions.avg_emissions as county_emissions,
income.'2019' as per_capita_income,
state_df_emissions.avg_emissions as state_emissions
FROM cdc_analysis
LEFT JOIN county_df_emissions on cdc_analysis.countyfips = county_df_emissions.GeoFips
LEFT JOIN income on cdc_analysis.countyfips = income.GeoFips
LEFT JOIN state_df_emissions on cdc_analysis.StateCode = state_df_emissions.state_code

"""
pysqldf = lambda q: sqldf(q, globals())
master_df = pysqldf(q)

In [15]:
# Fill county's with NA PM25 emissions data with average State PM25 emissions
master_df['county_emissions'] = master_df['county_emissions'].fillna(master_df['state_emissions'])

In [16]:
master_df.shape

(3142, 13)

#### Data diagnostics

Check for missing values in columns of interest

In [17]:
# Explanatory Variables
print(np.sum(master_df.county_emissions.isnull()))
print(np.sum(master_df.csmoking_adjprev.isnull()))
print(np.sum(master_df.access2_adjprev.isnull()))
print(np.sum(master_df.per_capita_income.isnull()))

0
21
21
53


In [18]:
# Dependent Variable
print(np.sum(master_df.casthma_adjprev.isnull()))

21


In [19]:
# print rows that are NA
master_df[master_df.access2_adjprev.isna()]

,stateabbr,statedesc,countyname,countyfips,totalpopulation,access2_adjprev,casthma_adjprev,csmoking_adjprev,StateCode,CountyCode,county_emissions,per_capita_income,state_emissions
199,NJ,New Jersey,Union,34039,556341,NaN,NaN,NaN,34,039,8.583793,68867.0,7.86686
205,NJ,New Jersey,Cumberland,34011,149527,NaN,NaN,NaN,34,011,7.798476,41327.0,7.86686
221,NJ,New Jersey,Ocean,34029,607186,NaN,NaN,NaN,34,029,6.259420,53517.0,7.86686
224,NJ,New Jersey,Essex,34013,798975,NaN,NaN,NaN,34,013,8.408746,65927.0,7.86686
322,NJ,New Jersey,Hudson,34017,672391,NaN,NaN,NaN,34,017,8.140014,67570.0,7.86686
327,NJ,New Jersey,Passaic,34031,501826,NaN,NaN,NaN,34,031,7.673451,51203.0,7.86686
402,NJ,New Jersey,Sussex,34037,140488,NaN,NaN,NaN,34,037,7.866860,64284.0,7.86686
553,NJ,New Jersey,Cape May,34009,92039,NaN,NaN,NaN,34,009,7.866860,63203.0,7.86686
815,NJ,New Jersey,Hunterdon,34019,124371,NaN,NaN,NaN,34,019,7.785955,91946.0,7.86686
1008,NJ,New Jersey,Morris,34027,491845,NaN,NaN,NaN,34,027,5.553097,99140.0,7.86686


To fill in missing values we will aggregate the missing features by state and input those values at the county level

In [20]:
# write query
q = """SELECT statedesc, 
StateCode,
AVG(csmoking_adjprev) as csmoking_adjprev_state,
AVG(access2_adjprev) as access2_adjprev_state,
AVG(per_capita_income) as per_capita_income_state,
AVG(casthma_adjprev) as casthma_adjprev_state
FROM master_df
GROUP BY statedesc
"""
pysqldf = lambda q: sqldf(q, globals())
state_agg = pysqldf(q)

Join state_agg dataframe onto master dataframe

In [21]:
# write query
q = """SELECT master_df.*,
state_agg.csmoking_adjprev_state,
state_agg.access2_adjprev_state,
state_agg.per_capita_income_state,
state_agg.casthma_adjprev_state
FROM master_df
LEFT JOIN state_agg on master_df.StateCode = state_agg.StateCode
"""
pysqldf = lambda q: sqldf(q, globals())
master_df = pysqldf(q)

In [22]:
master_df.head()

,stateabbr,statedesc,countyname,countyfips,totalpopulation,access2_adjprev,casthma_adjprev,csmoking_adjprev,StateCode,CountyCode,county_emissions,per_capita_income,state_emissions,csmoking_adjprev_state,access2_adjprev_state,per_capita_income_state,casthma_adjprev_state
0,NC,North Carolina,Warren,37185,19731,23.3,10.9,24.6,37,185,7.565971,30317.0,7.565971,21.27500,20.399000,41171.100000,9.768
1,SC,South Carolina,Aiken,45003,170872,19.7,9.9,18.3,45,003,7.449686,44503.0,7.449686,21.21087,21.095652,39970.282609,10.250
2,SC,South Carolina,Calhoun,45017,14553,20.0,10.3,20.0,45,017,7.449686,41620.0,7.449686,21.21087,21.095652,39970.282609,10.250
3,SC,South Carolina,Darlington,45031,66618,21.0,10.9,22.9,45,031,7.449686,39085.0,7.449686,21.21087,21.095652,39970.282609,10.250
4,SC,South Carolina,Florence,45041,138293,19.8,10.3,20.4,45,041,7.714407,43759.0,7.449686,21.21087,21.095652,39970.282609,10.250


Input missing values

In [23]:
# Fill county's with NA average smoking rates data with average State  rates 
master_df['csmoking_adjprev'] = master_df['csmoking_adjprev'].fillna(master_df['csmoking_adjprev_state'])

# Fill county's with NA average accessibility to health care data with average State rates 
master_df['access2_adjprev'] = master_df['access2_adjprev'].fillna(master_df['access2_adjprev_state'])

# Fill county's with NA per capita income data with average State rates 
master_df['per_capita_income'] = master_df['per_capita_income'].fillna(master_df['per_capita_income_state'])

# Fill county's with NA average asthma rates data with average State rates 
master_df['casthma_adjprev'] = master_df['casthma_adjprev'].fillna(master_df['casthma_adjprev_state'])

Check for missing values again

In [24]:
# Explanatory Variables
print(np.sum(master_df.county_emissions.isnull()))
print(np.sum(master_df.csmoking_adjprev.isnull()))
print(np.sum(master_df.access2_adjprev.isnull()))
print(np.sum(master_df.per_capita_income.isnull()))

0
21
21
0


In [25]:
# Dependent Variable
print(np.sum(master_df.casthma_adjprev.isnull()))

21


In [26]:
# print rows that are NA
master_df[master_df.access2_adjprev.isna()]

,stateabbr,statedesc,countyname,countyfips,totalpopulation,access2_adjprev,casthma_adjprev,csmoking_adjprev,StateCode,CountyCode,county_emissions,per_capita_income,state_emissions,csmoking_adjprev_state,access2_adjprev_state,per_capita_income_state,casthma_adjprev_state
199,NJ,New Jersey,Union,34039,556341,NaN,NaN,NaN,34,039,8.583793,68867.0,7.86686,NaN,NaN,67401.904762,NaN
205,NJ,New Jersey,Cumberland,34011,149527,NaN,NaN,NaN,34,011,7.798476,41327.0,7.86686,NaN,NaN,67401.904762,NaN
221,NJ,New Jersey,Ocean,34029,607186,NaN,NaN,NaN,34,029,6.259420,53517.0,7.86686,NaN,NaN,67401.904762,NaN
224,NJ,New Jersey,Essex,34013,798975,NaN,NaN,NaN,34,013,8.408746,65927.0,7.86686,NaN,NaN,67401.904762,NaN
322,NJ,New Jersey,Hudson,34017,672391,NaN,NaN,NaN,34,017,8.140014,67570.0,7.86686,NaN,NaN,67401.904762,NaN
327,NJ,New Jersey,Passaic,34031,501826,NaN,NaN,NaN,34,031,7.673451,51203.0,7.86686,NaN,NaN,67401.904762,NaN
402,NJ,New Jersey,Sussex,34037,140488,NaN,NaN,NaN,34,037,7.866860,64284.0,7.86686,NaN,NaN,67401.904762,NaN
553,NJ,New Jersey,Cape May,34009,92039,NaN,NaN,NaN,34,009,7.866860,63203.0,7.86686,NaN,NaN,67401.904762,NaN
815,NJ,New Jersey,Hunterdon,34019,124371,NaN,NaN,NaN,34,019,7.785955,91946.0,7.86686,NaN,NaN,67401.904762,NaN
1008,NJ,New Jersey,Morris,34027,491845,NaN,NaN,NaN,34,027,5.553097,99140.0,7.86686,NaN,NaN,67401.904762,NaN


Appears New jersey has no CDC data for any fields. Because of this, lets exclude New Jersey from this analysis and focus on all other states (plus District of Columbia).

In [27]:
master_df = master_df[master_df.StateCode != '34']

In [33]:
dff = master_df.groupby(['statedesc']).agg({'totalpopulation':'sum'}).reset_index()

In [35]:
dff.dtypes

statedesc          object
totalpopulation     int64
dtype: object

## Regression Model

#### OLS Model

Import needed packages

In [31]:
import statsmodels.formula.api as sm
from scipy.stats import pearsonr
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

Summary statistics

In [32]:
master_df.describe()

,totalpopulation,access2_adjprev,casthma_adjprev,csmoking_adjprev,county_emissions,per_capita_income,state_emissions,csmoking_adjprev_state,access2_adjprev_state,per_capita_income_state,casthma_adjprev_state
count,3.121000e+03,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000,3121.000000
mean,1.023253e+05,18.373662,9.719449,20.397661,7.602723,45347.496808,7.667687,20.397661,18.373662,45347.496808,9.719449
std,3.328885e+05,6.839567,0.974270,4.188090,1.290314,12617.058481,1.155014,3.046591,5.910140,6186.590900,0.729419
min,8.600000e+01,7.900000,7.200000,6.500000,0.915000,18944.000000,2.597847,11.137931,10.040000,36175.670732,8.238710
25%,1.080300e+04,13.400000,9.000000,17.600000,7.014711,37826.000000,7.062724,18.418110,13.420482,39576.431579,9.103448
50%,2.547300e+04,16.600000,9.800000,20.100000,7.883339,43196.000000,8.075398,19.893750,17.773438,45259.794118,9.872932
75%,6.682400e+04,21.200000,10.400000,23.200000,8.575984,49652.000000,8.575984,22.454717,20.940000,48728.282051,10.312174
max,1.003911e+07,56.600000,14.300000,43.000000,13.098077,222893.000000,9.153661,26.709167,32.727165,80819.000000,11.132727


Produce correlation matrix to examine variables that might be highly correlated to avoid multi-colinearity

In [33]:
rho = master_df.corr()
rho

,totalpopulation,access2_adjprev,casthma_adjprev,csmoking_adjprev,county_emissions,per_capita_income,state_emissions,csmoking_adjprev_state,access2_adjprev_state,per_capita_income_state,casthma_adjprev_state
totalpopulation,1.000000,-0.007385,-0.157270,-0.278976,0.040587,0.247884,-0.008185,-0.133334,0.007258,0.128635,-0.032417
access2_adjprev,-0.007385,1.000000,-0.140265,0.100791,0.284465,-0.263310,0.285723,-0.035354,0.864110,-0.227580,-0.298012
casthma_adjprev,-0.157270,-0.140265,1.000000,0.703402,0.156154,-0.447619,0.178703,0.435854,-0.258204,-0.354687,0.748683
csmoking_adjprev,-0.278976,0.100791,0.703402,1.000000,0.290825,-0.595899,0.313997,0.727442,-0.029763,-0.502624,0.423488
county_emissions,0.040587,0.284465,0.156154,0.290825,1.000000,-0.230342,0.892766,0.400025,0.309638,-0.457157,0.209228
per_capita_income,0.247884,-0.263310,-0.447619,-0.595899,-0.230342,1.000000,-0.241577,-0.338796,-0.129139,0.490335,-0.232296
state_emissions,-0.008185,0.285723,0.178703,0.313997,0.892766,-0.241577,1.000000,0.431646,0.330655,-0.492678,0.238690
csmoking_adjprev_state,-0.133334,-0.035354,0.435854,0.727442,0.400025,-0.338796,0.431646,1.000000,-0.040914,-0.690948,0.582161
access2_adjprev_state,0.007258,0.864110,-0.258204,-0.029763,0.309638,-0.129139,0.330655,-0.040914,1.000000,-0.263369,-0.344878
per_capita_income_state,0.128635,-0.227580,-0.354687,-0.502624,-0.457157,0.490335,-0.492678,-0.690948,-0.263369,1.000000,-0.473749


In this next step, regular OLS will be conducted. To test for random effects a Breusch- Pagan Lagrange Multiplier Test will be conducted. Whereby the following hypothesis test will be performed:

𝑯𝟎: 𝑽𝒂𝒓𝒊𝒂𝒏𝒄𝒆 𝒂𝒄𝒓𝒐𝒔𝒔 𝒐𝒃𝒔𝒆𝒓𝒗𝒂𝒕𝒊𝒐𝒏𝒔 𝒊𝒔 𝒆𝒒𝒖𝒂𝒍 𝒕𝒐 𝒛𝒆𝒓𝒐 (𝑶𝑳𝑺 𝒊𝒔 𝒂𝒑𝒑𝒓𝒐𝒑𝒓𝒊𝒂𝒕𝒆)

𝑯𝟏: 𝑽𝒂𝒓𝒊𝒂𝒏𝒄𝒆 𝒂𝒄𝒓𝒐𝒔𝒔 𝒐𝒃𝒔𝒆𝒓𝒗𝒂𝒕𝒊𝒐𝒏𝒔 𝒊𝒔 𝒏𝒐𝒕 𝒆𝒒𝒖𝒂𝒍 𝒕𝒐 𝒛𝒆𝒓𝒐 (𝑹𝒂𝒏𝒅𝒐𝒎 𝑬𝒇𝒇𝒆𝒄𝒕𝒔 𝒊𝒔 𝒂𝒑𝒑𝒓𝒐𝒑𝒓𝒊𝒂𝒕𝒆)

In [34]:
normal_ols = sm.ols(formula='''casthma_adjprev ~ np.log(county_emissions) + csmoking_adjprev +
                    access2_adjprev + np.log(per_capita_income)''',
                          data=master_df).fit()
print(normal_ols.summary())

                            OLS Regression Results                            
Dep. Variable:        casthma_adjprev   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     966.3
Date:                Sun, 09 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:37:48   Log-Likelihood:                -3087.8
No. Observations:                3121   AIC:                             6186.
Df Residuals:                    3116   BIC:                             6216.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [35]:
#perform Bresuch-Pagan test
names = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(normal_ols.resid, normal_ols.model.exog)

lzip(names, test)

[('Lagrange multiplier statistic', 124.35394974713637),
 ('p-value', 6.272515418886365e-26),
 ('f-value', 32.3267163450435),
 ('f p-value', 1.9418323653136825e-26)]

We can reject the null hypothesis and conclude that Random Effects are present. Therefore a Fixed Effects model with States as dummy variables is appropriate. In this previous regression model the coefficient for county emissions was not significant. Since we are running a Fixed Effects model to account for state specific effects, perhaps it is more appropriate to examine the impact of state emissions on asthma rates. Because emissions are bound by no geographical location, this might be a more appropriate approach. Addiitonally, State governments might have more authority to impliment measures to reduce overall PM25 emission rates.

#### Fixed Effects

In [36]:
fixed_model = sm.ols(formula='''casthma_adjprev ~ np.log(state_emissions) + csmoking_adjprev +
                    access2_adjprev + np.log(per_capita_income) + C(statedesc)''',
                          data=master_df).fit()
print(fixed_model.summary())

                            OLS Regression Results                            
Dep. Variable:        casthma_adjprev   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     433.0
Date:                Sun, 09 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:37:50   Log-Likelihood:                -1036.8
No. Observations:                3121   AIC:                             2180.
Df Residuals:                    3068   BIC:                             2500.
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


Obtain regression coefficients

In [37]:
# State Emissions
state_emissions_coef = pd.read_html(fixed_model.summary2().as_html())[1][1][51]
state_emissions_coef = f = float(state_emissions_coef)
print("State emissions regression coefficient = {}.".format(state_emissions_coef))

# Smoking Rates
csmoking_adjprev_coef = pd.read_html(fixed_model.summary2().as_html())[1][1][52]
csmoking_adjprev_coef = f = float(csmoking_adjprev_coef)
print("Smoking rates regression coefficient = {}.".format(csmoking_adjprev_coef))

# No health care accessibility
access2_adjprev_coef = pd.read_html(fixed_model.summary2().as_html())[1][1][53]
access2_adjprev_coef = f = float(access2_adjprev_coef)
print("No health care accessibility regression coefficient = {}.".format(access2_adjprev_coef))

State emissions regression coefficient = 0.4377.
Smoking rates regression coefficient = 0.2034.
No health care accessibility regression coefficient = -0.0118.


## Analysis of Findings

Now that we have found the relationship between various factors and their impact on Asthma rates, lets examine the societal benefits of a 1% reduction in state emission and smoking rates, while examing the impacts of a 1% increase in health care accessibility. based on the following article, the societal cost of asthma per person is approximately $3,100 USD per year (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5219738/). Reducing overall asthma rates would have a great economic benefit for many households. Because asthma has no permanent cure our focus will be on prevention of future asthma rates, with a focus on how reductions today could impact the population (age 19-64) by 2030 based on birth rate data from the United States Census Bureau (https://www.census.gov/library/publications/2020/demo/p25-1144.html & https://www.kff.org/other/state-indicator/distribution-by-age/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

The 2030 projected United States population is 355.1 million, with approximately 60% of the population falling within the age range of 19-64. Noting that the 2020 popultion is 332.6 we can assess the growth rate over this approximate period. Using this information lets find the current (2019) population and examine the growth rate, which can be applied uniformly to each respective county

In [38]:
growth_rate = (355.1-332.6) / 332.6
growth_rate_percent = "{0:.2f}%". format(growth_rate*100)
print("The expected population growth between 2020 and 2030 for the United States is = {}.".format(growth_rate_percent))

The expected population growth between 2020 and 2030 for the United States is = 6.76%.


In [39]:
# Apply growth rate to population within ragne of analysis (19-64)
master_df['net_growth_19to64'] = (((master_df['totalpopulation']*0.6) * (1 + growth_rate)) - (master_df['totalpopulation']*0.6))
master_df['net_growth_19to64'] = master_df['net_growth_19to64'].astype(int)

Impact of a 1% reduction in State emissions 

In [40]:
# Impact of reduced emissions

# Set percentage impact
x = 1

# Estimated number of reduced asthma cases by county
reduced_emissions = (master_df['net_growth_19to64'] * (master_df['casthma_adjprev']/100)) - (master_df['net_growth_19to64'] * ((master_df['casthma_adjprev'] - (state_emissions_coef*x))/100))
# Convert to int
reduced_emissions = reduced_emissions.astype(int)
# Total impact
reduced_emissions = np.sum(reduced_emissions)
# Apply monetary societal benefit
reduced_emissions_monetary =  reduced_emissions * 3100
# Convert to int
reduced_emissions_monetary = reduced_emissions_monetary.astype(int)
# Convert to monetary object
reduced_emissions_monetary = "${0:,.0f}". format(reduced_emissions_monetary)

string = "A %s percent reduction in State emissions is associated with an estimated %s reduced asthma cases which amounts to %s in total societal beneftis" % (x, reduced_emissions, reduced_emissions_monetary)
print(string)

A 1 percent reduction in State emissions is associated with an estimated 55182 reduced asthma cases which amounts to $171,064,200 in total societal beneftis


Impact of a 1% reduction in Smoking Rates

In [41]:
# Impact of reduced smoking rates

# Set percentage impact
x = 1

# Estimated number of reduced asthma cases by county
reduced_smoking = (master_df['net_growth_19to64'] * (master_df['casthma_adjprev']/100)) - (master_df['net_growth_19to64'] * ((master_df['casthma_adjprev'] - (csmoking_adjprev_coef*x))/100))
# Convert to int
reduced_smoking = reduced_smoking.astype(int)
# Total impact
reduced_smoking = np.sum(reduced_smoking)
# Apply monetary societal benefit
reduced_smoking_monetary = reduced_smoking * 3100
# Convert to int
reduced_smoking_monetary = reduced_smoking_monetary.astype(int)
# Convert to monetary object
reduced_smoking_monetary = "${0:,.0f}". format(reduced_smoking_monetary)

string = "A %s percent reduction in County smoking rates is associated with an estimated %s reduced asthma cases which amounts to %s in total societal beneftis" % (x, reduced_smoking, reduced_smoking_monetary)
print(string)

A 1 percent reduction in County smoking rates is associated with an estimated 24863 reduced asthma cases which amounts to $77,075,300 in total societal beneftis


Impact of 1% increase in Health Care Accessibility

In [48]:
# Impact of increased health care accessibility

# Set percentage impact
x = 1

# Estimated number of reduced asthma cases by county
increased_access = (master_df['net_growth_19to64'] * (master_df['casthma_adjprev']/100)) - (master_df['net_growth_19to64'] * ((master_df['casthma_adjprev'] - (-access2_adjprev_coef*x))/100))
# Convert to int
increased_access = increased_access.astype(int)
# Total impact
increased_access = np.sum(increased_access)
# Apply monetary societal benefit
increased_access_monetary = increased_access * 3100
# Convert to int
increased_access_monetary = increased_access_monetary.astype(int)
# Convert to monetary object
increased_access_monetary = "${0:,.0f}". format(increased_access_monetary)

string = "A %s percent increase in health care accessibility, by County, is associated with an estimated %s reduced asthma cases which amounts to %s in total societal beneftis" % (x, increased_access, increased_access_monetary)
print(string)

A 1 percent increase in health care accessibility, by County, will result in an estimated 891 reduction in asthma cases which amounts to $2,762,100 in total societal beneftis


## Extension of Model

Many great insights have come from this analysis. However the interactivity of this analysis is quite limited. For further extension into Analysis of Findings the data and model from this analysis will be utilized as Plotly objects in developing a Dash web interface.

In [42]:
# Export Master Data
master_df.to_csv("Data/Master_Data.csv", index=False)